In [441]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [442]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [443]:
n_core = 10
N = 256/2
print(list(linspace(1,N+1-N//n_core,n_core,dtype='int')))
print(list(linspace(N//n_core,N,n_core,dtype='int')))

n_dt = 5000

# integer          , parameter :: ia(ni)=[1,257,513,769]
# integer          , parameter :: ib(ni)=[256,512,768,1024]

[1, 13, 26, 39, 52, 65, 78, 91, 104, 117]
[12, 24, 37, 50, 63, 76, 89, 102, 115, 128]


In [444]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [445]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [451]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20221006' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220930' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
#     print(i,j)
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
#         print(k,l)
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'Temp_SimuType0' in l and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
all_subdir.sort()
all_Temp.sort()
all_xva_3DHarmo.sort()
all_xva_start.sort()
all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')
print(all_subdir)

['/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC05_RF05/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC06_RF06/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC07_RF07/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC08_RF08/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC09_RF09/Try00']


In [453]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC05_RF05
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC05_RF05
DC06_RF06
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC06_RF06
DC07_RF07
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC07_RF07
DC08_RF08
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC08_RF08


IndexError: list index out of range

In [516]:
def load_xyz_init_bin_DP(str_load):
    fid = open(str_load+'.bin',  'rb')   
    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]     

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    dt   = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag
    
    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    iRF  = fromfile(fid,  int32, 3)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    print(f)
    save_xva = fromfile(fid,  float64, 12*N_ions)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag
    print(junk)

    fid.close
    save_xva = reshape(save_xva, (12,N_ions), order='F')

    return save_xva

def plot_XYZ(file_name,fig_name='2',fig_title='XYZ'):
    save_xva = load_xyz_init_bin_DP(file_name)
    r_LC = save_xva[0:3,:] 
    v_LC = save_xva[3:6,:]
    a_LC = save_xva[6:9,:]
    v_LC_avg = save_xva[9:,:]
    print(r_LC)
    figure(fig_name); clf()
    title(fig_title)
    subplot(211)
    plot(r_LC [0,:]*1e6,r_LC [1,:]*1e6,'8',color='xkcd:purplish blue')
    xlabel('x [µm]')
    ylabel('y [µm]')
    grid()

    # subplot(212,aspect=1.0)
    subplot(212,aspect=1.0)
    plot(r_LC [2,:]*1e6,r_LC [1,:]*1e6,'8',color='xkcd:purplish blue')
    xlabel('z [µm]')
    ylabel('y [µm]')
    grid()
    
    tight_layout()

def load_xva_init_bin_Lan(str_load, flag_print):      
    fid = open(str_load+'.bin',  'rb')   
    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]     

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    dt   = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag
    
    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    iRF  = fromfile(fid,  int32, 3)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    print(f)
    save_xva = fromfile(fid,  float64, 12*N_ions)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag
    print(junk)

    fid.close
    save_xva = reshape(save_xva, (12,N_ions), order='F')

    return N_ions, j_save, dt, iRF, save_xva

In [517]:
# address = all_subdir[0]
# print(address)
# my_file = 'xva_3D_Harmo_N1024_T01000uK_F0.50D-19Kg_s_5'
# print(address+'/'+my_file)
# N_ions, j_save, dt, iRF, save_xva = load_xva_init_bin_Lan(address+'/'+my_file,1)

# print(N_ions, j_save, dt, iRF, save_xva)
# figure('XYZ_Lan',clear=True)
# ax = subplot(111)
# ax.plot(save_xva[0],save_xva[1],marker='.',ls='')
# ax.grid()
# ax.set_aspect('equal')

plot_XYZ(address+'/xva_start_SimuType0_N01024_Vrf0055_Udc0.7928D+01V_nt1000',
         fig_name='2',fig_title='XYZ')

[98304]
[98304]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.18323180e-05 -4.03081013e-05 -5.66392728e-05 ... -3.63880998e-06
   2.48691839e-06  1.61542457e-05]
 [-3.58423720e-05  2.06557802e-06 -7.68712401e-05 ...  2.35443288e-05
   6.41459681e-05  4.79419551e-05]]


In [8]:
j = 1 # all_subdir[3]
str_N  = all_Temp[j].find('_N')
cond_N = all_Temp[j][str_N+3:str_N+7]
str_v  = all_subdir[j].find('DC')
cond   = all_Temp[j][str_v:str_v+9]
print(cond)
outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)

data = loadtxt(all_Temp[j])
time_s = data[:,0]
T_CM = data[:,1:4]
T_aux = data[:,4:]
T_aux_avg = mean(T_aux,axis=1)
r2_v2 = load_xv2rlim(all_trjN[j])
#r2    = r2_v2[0:3,:]
#v2    = r2_v2[3:6,:]
#rlim  = r2_v2[6:9,:]

np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
print(outfile) # , r2_v2_rlim=r2_v2

DC04_RF04
[101524680]
[101524680]
Time_and_temp_RFHEAT_N0512_DC04_RF04


# With extended

In [335]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220930' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_Temp_extended = []
all_trjN = []
all_trjN_extended = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
#     print(i)
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
            all_trjN_extended.append('')
        if 'TrjN' in l and 'extended' in l:
            all_trjN_extended[i-2] = f'{j[0]}/{l}'
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'Temp_SimuType0' in l and '.dat' in l and not 'extended' in l:
            all_Temp.append(j[0]+'/'+l)
            all_Temp_extended.append('')
        if 'Temp_SimuType0' in l and '.dat' in l and 'extended' in l:
            all_Temp_extended[i-2] = f'{j[0]}/{l}'
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
# all_subdir.sort()
# all_Temp.sort()
# all_Temp_extended.sort()
# all_xva_3DHarmo.sort()
# all_xva_start.sort()
# all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [322]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'xxxxTime_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    try:
        data_extended = loadtxt(all_Temp_extended[j])
        time_s = hstack((data[:,0],data_extended[:,0]))
        T_CM = hstack((data[:,1:4],data_extended[:,1:4]))
        T_aux = hstack((data[:,4:],data_extended[:,4:]))
        T_aux_avg = mean(T_aux,axis=1)
        r2_v2 = load_xv2rlim(all_trjN[j])
        r2_v2_extended = load_xv2rlim(all_trjN_extended[j])
        r2_v2 = hstack((r2_v2 ,r2_v2_extended))
        print('coucou')
    except:
        time_s = data[:,0]
        T_CM = data[:,1:4]
        T_aux = data[:,4:]
        T_aux_avg = mean(T_aux,axis=1)
        r2_v2 = load_xv2rlim(all_trjN[j])
        print('pas coucou')
        #r2    = r2_v2[0:3,:]
        #v2    = r2_v2[3:6,:]
        #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC04_RF04
[101524680]
[101524680]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04
DC04_RF04
[101524680]
[101524680]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04
DC04_RF04
[50762304]
[50762304]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04


In [325]:
all_subdir[1]

'/home/adrien/RemoteFS/Rivendel/Simulations/20220930/DC04_RF04/Try01'

In [438]:
j = 1
print(all_subdir[j])
str_N  = all_Temp[j].find('_N')
cond_N = all_Temp[j][str_N+3:str_N+7]
str_v  = all_subdir[j].find('DC')
cond   = all_Temp[j][str_v:str_v+9]
print(cond)
outfile = 'xxxxstackTime_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)

data = loadtxt(all_Temp[j])
try:
    data_extended = loadtxt(all_Temp_extended[j])
    time_s = hstack((data[:,0],array(data_extended[:,0])+max(data[:,0])-min(data[:,0])))
    T_CM = vstack((data[:,1:4],data_extended[:,1:4]))
    T_aux = vstack((data[:,4:],data_extended[:,4:]))
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    r2_v2_extended = load_xv2rlim(all_trjN_extended[j])
    r2_v2 = hstack((r2_v2 ,r2_v2_extended))
    print('coucou')
except:
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    print('pas coucou')
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]

np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
print(outfile) # , r2_v2_rlim=r2_v2
print('fini')

/home/adrien/RemoteFS/Rivendel/Simulations/20220930/DC04_RF04/Try01
DC04_RF04
[101524680]
[101524680]
[50762304]
[50762304]
coucou
xxxxstackTime_and_temp_RFHEAT_N0512_DC04_RF04
fini


array([0.00100062, 0.00100112, 0.00100162, ..., 0.35351512, 0.35351562,
       0.35351612])

In [430]:
loadtxt(all_Temp[1])[0],time_s_extended

(array([1.00062450e-03, 1.03382074e-05, 1.37773710e-05, 7.12058867e-05,
        6.20725325e-03, 1.01597441e-02, 9.23096771e-03]),
 array([1.00062450e-03, 1.03382074e-05, 1.37773710e-05, 7.12058867e-05,
        6.20725325e-03, 1.01597441e-02, 9.23096771e-03]))

In [416]:
T_aux = vstack((data[:,4:],data_extended[:,4:]))
shape(T_aux)
shape(mean(T_aux,axis=1))

(2115097,)